In [9]:
import torch
import os
import glob
import pandas as pd
import datetime
import requests
from detection import category
import private

In [3]:
checkpoints = ['yolov5m', 'yolov5x6']
print(type(checkpoints))

<class 'list'>


In [4]:
folder_path = 'data/'
image_files = glob.glob(os.path.join(folder_path, '*.png'))

In [10]:
for checkpoint in checkpoints:
    model = torch.hub.load('ultralytics/yolov5', checkpoint) # model selection
    for image_file in image_files:
        results = model(image_file) # image selection
        #results.print()
        #results.show()
        
        # drop word 'png'
        exclude_word = ["png"]
        image_path = image_file
        image_file = [image_file for image_file in image_file.split('.') if image_file != exclude_word]
        
        # split camera name
        camera_name = image_file[0]
        camera_name = [camera_name.replace("\\", "/")]
        
        # drop word 'data'
        exclude_word = ["data"]
        camera_name = camera_name[0]
        camera_name_parts = camera_name.split('/')
        filtered_parts = [part for part in camera_name_parts if part not in exclude_word]
        final_camera_name = "_".join(filtered_parts)
        
        #drop word '_daytime'
        final_camera_name = final_camera_name.replace("_daytime", "")
        #drop word '_night'
        final_camera_name = final_camera_name.replace("_night", "")
        
        object_coords = results.pandas().xyxy[0] # object detection results
        object_coords.rename(columns={'class':'type_class'}, inplace=True)
        
        # add camera_name column into dataframe
        object_coords['camera_name'] = final_camera_name
        col1=object_coords.columns[-1]
        col2=object_coords.columns[:-1]
        col2=col2.insert(0, col1)
        object_coords=object_coords[col2]
        
        # load img saved time
        if os.path.exists(image_path):
            timestamp = os.path.getmtime(image_path)
            modified_time = datetime.datetime.fromtimestamp(timestamp)
            modified_time = modified_time.strftime("%Y-%m-%d %H:%M:%S")
            print(f"Image '{image_path}' was last modified at: {modified_time}")
            
            object_coords['time'] = modified_time
            col1=object_coords.columns[-1]
            col2=object_coords.columns[:-1]
            col2=col2.insert(1, col1)
            object_coords=object_coords[col2]
        else:
            print(f"Image '{image_path}' does not exist.")
            
        # drop boundary box coordinates data
        exclude_index = ['xmax', 'xmin', 'ymax', 'ymin', 'confidence']
        for index in exclude_index:
            #print('drop ' + index)
            object_coords = object_coords.drop(columns=index)
            
        object_coords_detection = category(object_coords)
        
        for i in range(len(object_coords_detection)):
            if object_coords_detection['action'][i] == 'person detected':
                # save to json
                save_file_path = 'results/xlsx'+image_file[0]+'_'+checkpoint+'person_results.json'
                object_coords_detection.to_json(save_file_path, orient = 'records')
                json_data = object_coords_detection.to_dict(orient = 'records')
                
                #PUT code for client
                url = private.url
                
                for j in range(len(json_data)):
                    print(json_data[j])
                    response = requests.put(url, json=json_data[j])

                    if response.status_code == 200:
                        print("Success:", response.json())
                    else:
                        print("Error:", response.status_code, response.text)

Using cache found in /Users/bongeungu/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-8-5 Python-3.11.4 torch-2.0.1 CPU

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients
Adding AutoShape... 


Image 'data/Je1sunhwan_soraetunnel_night.png' was last modified at: 2023-08-11 13:08:09
Image 'data/EunhangIntersection_night.png' was last modified at: 2023-08-11 13:08:09
Image 'data/DaeyaIntersection_daytime.png' was last modified at: 2023-08-11 13:08:09
Image 'data/Je1sunhwan_soraetunnel_daytime.png' was last modified at: 2023-08-11 13:08:09
Image 'data/SincheonIntersection_night.png' was last modified at: 2023-08-11 13:08:09
Image 'data/DaeyaIntersection_night.png' was last modified at: 2023-08-11 13:08:09
Image 'data/test.png' was last modified at: 2023-08-11 11:20:21
{'camera_name': 'test', 'time': '2023-08-11 11:20:21', 'type_class': 0, 'name': 'person', 'action': 'person detected'}


/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action'][i] = 'person detected'


Success: {'message': 'Item added successfully'}
{'camera_name': 'test', 'time': '2023-08-11 11:20:21', 'type_class': 2, 'name': 'car', 'action': 'NaN'}
Success: {'message': 'Item added successfully'}
{'camera_name': 'test', 'time': '2023-08-11 11:20:21', 'type_class': 2, 'name': 'car', 'action': 'NaN'}
Success: {'message': 'Item added successfully'}
{'camera_name': 'test', 'time': '2023-08-11 11:20:21', 'type_class': 2, 'name': 'car', 'action': 'NaN'}
Success: {'message': 'Item added successfully'}
{'camera_name': 'test', 'time': '2023-08-11 11:20:21', 'type_class': 2, 'name': 'car', 'action': 'NaN'}
Success: {'message': 'Item added successfully'}
{'camera_name': 'test', 'time': '2023-08-11 11:20:21', 'type_class': 2, 'name': 'car', 'action': 'NaN'}
Success: {'message': 'Item added successfully'}
{'camera_name': 'test', 'time': '2023-08-11 11:20:21', 'type_class': 2, 'name': 'car', 'action': 'NaN'}
Success: {'message': 'Item added successfully'}
{'camera_name': 'test', 'time': '2023-08

Using cache found in /Users/bongeungu/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-8-5 Python-3.11.4 torch-2.0.1 CPU



...

Image 'data/Je1sunhwan_soraetunnel_night.png' was last modified at: 2023-08-11 13:08:09
Image 'data/EunhangIntersection_night.png' was last modified at: 2023-08-11 13:08:09
{'camera_name': 'EunhangIntersection', 'time': '2023-08-11 13:08:09', 'type_class': 5, 'name': 'bus', 'action': 'NaN'}


/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action'][i] = 'person detected'
/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action'][i] = 'person detected'


Success: {'message': 'Item added successfully'}
{'camera_name': 'EunhangIntersection', 'time': '2023-08-11 13:08:09', 'type_class': 2, 'name': 'car', 'action': 'NaN'}
Success: {'message': 'Item added successfully'}
{'camera_name': 'EunhangIntersection', 'time': '2023-08-11 13:08:09', 'type_class': 2, 'name': 'car', 'action': 'NaN'}
Success: {'message': 'Item added successfully'}
{'camera_name': 'EunhangIntersection', 'time': '2023-08-11 13:08:09', 'type_class': 2, 'name': 'car', 'action': 'NaN'}
Success: {'message': 'Item added successfully'}
{'camera_name': 'EunhangIntersection', 'time': '2023-08-11 13:08:09', 'type_class': 2, 'name': 'car', 'action': 'NaN'}
Success: {'message': 'Item added successfully'}
{'camera_name': 'EunhangIntersection', 'time': '2023-08-11 13:08:09', 'type_class': 2, 'name': 'car', 'action': 'NaN'}
Success: {'message': 'Item added successfully'}
{'camera_name': 'EunhangIntersection', 'time': '2023-08-11 13:08:09', 'type_class': 9, 'name': 'traffic light', 'acti

/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action'][i] = 'person detected'
/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/detection.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  object_coords['action'][i] = 'person detected'


Success: {'message': 'Item added successfully'}
{'camera_name': 'test', 'time': '2023-08-11 11:20:21', 'type_class': 2, 'name': 'car', 'action': 'NaN'}
Success: {'message': 'Item added successfully'}
{'camera_name': 'test', 'time': '2023-08-11 11:20:21', 'type_class': 2, 'name': 'car', 'action': 'NaN'}
Success: {'message': 'Item added successfully'}
{'camera_name': 'test', 'time': '2023-08-11 11:20:21', 'type_class': 7, 'name': 'truck', 'action': 'NaN'}
Success: {'message': 'Item added successfully'}
{'camera_name': 'test', 'time': '2023-08-11 11:20:21', 'type_class': 2, 'name': 'car', 'action': 'NaN'}
Success: {'message': 'Item added successfully'}
{'camera_name': 'test', 'time': '2023-08-11 11:20:21', 'type_class': 2, 'name': 'car', 'action': 'NaN'}
Success: {'message': 'Item added successfully'}
{'camera_name': 'test', 'time': '2023-08-11 11:20:21', 'type_class': 2, 'name': 'car', 'action': 'NaN'}
Success: {'message': 'Item added successfully'}
{'camera_name': 'test', 'time': '2023-